# Instructions
This notebook was executed on Kaggle with a P100 GPU.

You can find more about this work in https://medium.com/@momayiz.imane/inviting-stable-diffusion-model-to-morocco-edf91a764458

In [ ]:
!git clone https://github.com/huggingface/diffusers
!pip install -q /kaggle/working/diffusers/.
!pip install -q -r /kaggle/working/diffusers/examples/dreambooth/requirements.txt
!pip install -q bitsandbytes

In [ ]:
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch

In [ ]:
!pip install numpy==1.22.0

In [ ]:
import os 
os.makedirs('/kaggle/working/output/pretrained/', exist_ok=True)
os.makedirs('/kaggle/working/output/finetuned/', exist_ok=True)
os.makedirs("/kaggle/model", exist_ok=True)

# Pre-trained model

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5").to("cuda")

In [ ]:
num_inference_steps = 30
guidance_scale = 10
image = pipeline("a moroccan city",
                 num_inference_steps=num_inference_steps,
                 guidance_scale=guidance_scale).images[0]
image

# Finetuning with DreamBooth

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "/kaggle/working/data"
snapshot_download(
    "imomayiz/morocco-img",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config()

In [ ]:
! accelerate launch /kaggle/working/diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="/kaggle/working/data/data/cities" \
  --output_dir="/kaggle/model" \
  --instance_prompt="a moroccan city" \
  --gradient_checkpointing \
  --use_8bit_adam \
  --resolution=512 \
  --train_batch_size=8 \
  --gradient_accumulation_steps=1 \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 

In [ ]:
unet = UNet2DConditionModel.from_pretrained("/kaggle/model/unet")

# if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
text_encoder = CLIPTextModel.from_pretrained("/kaggle/model/text_encoder")

ft_pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", unet=unet, text_encoder=text_encoder, dtype=torch.float16,
).to("cuda")

In [ ]:
num_inference_steps = 50
guidance_scale = 7.5
prompt = "a modern moroccan city"
image = ft_pipeline(prompt,
                 num_inference_steps=num_inference_steps,
                 guidance_scale=guidance_scale).images[0]
image

In [ ]:
image

In [ ]:
image.save(f"/kaggle/working/{prompt}_{num_inference_steps}_{guidance_scale}_3.jpg")